# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("../output_data/cities.csv")
weather_data

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations_df = weather_data[["Lat", "Lng"]].astype(float)

humidity_df = weather_data["Humidity"].astype(float)

In [4]:
# Plot Heatmap
fig = gmaps.figure()
max_intensity = np.max(humidity_df)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations_df, weights=humidity_df, dissipating=False, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [5]:
max_intensity

290.0

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
# Narrow the cities with wind speed less yhaan 15 mph, cloudiness less than/equal to 10 and temperature range of 70-85

narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 8) & (weather_data["Cloudiness"] <= 5) & 
                                    (weather_data["Max Temp"] >= 75) & (weather_data["Max Temp"] <= 85)].dropna()

narrowed_city_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
88,88,cidreira,0,BR,1558378770,66,-30.17,-50.22,76.56,5.50
125,125,arraial do cabo,0,BR,1558378491,78,-22.97,-42.02,84.20,4.70
139,139,vaini,0,IN,1558378780,88,15.34,74.49,78.90,1.59
176,176,nador,0,MA,1558378499,40,35.17,-2.93,75.20,6.93
268,268,dingle,4,PH,1558378836,80,11.00,122.67,79.08,2.73
494,494,baylo,1,PH,1558378889,69,7.85,126.06,77.00,1.07
518,518,bonito,0,BR,1558378893,49,-21.13,-56.49,84.30,7.72
527,527,tomatlan,0,MX,1558378894,59,19.93,-105.25,84.66,7.90


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
# Create a hotel DataFrame
hotel_df = narrowed_city_df.loc[:,["City", "Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
88,cidreira,BR,-30.17,-50.22,
125,arraial do cabo,BR,-22.97,-42.02,
139,vaini,IN,15.34,74.49,
176,nador,MA,35.17,-2.93,
268,dingle,PH,11.00,122.67,
494,baylo,PH,7.85,126.06,
518,bonito,BR,-21.13,-56.49,
527,tomatlan,MX,19.93,-105.25,


In [10]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type": "hotel",
             "keyword": "hotel",
             "radius": 5000,
             "key": g_key}

In [11]:
for index, row in hotel_df.iterrows():
    # Get city name, lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # Add keyword to params dict
    params["location"] = f"{lat},{lng}"
    
    # Assemble url and make API request
    print(f"Retrieving results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # Extract results
    results = response["results"]
    
    # Save the hotel name to DataFrame
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']
        
    # If no hotel is available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("--------------------------")
    
    # Wait 1 second to make another API request to avoid SSL Error
    time.sleep(1)
    
# Print end of search after complete
print("-------End of Search------")

Retrieving results for Index 88: cidreira.
Closest hotel in cidreira is Hotel Castelo.
--------------------------
Retrieving results for Index 125: arraial do cabo.
Closest hotel in arraial do cabo is Ocean View Hotel.
--------------------------
Retrieving results for Index 139: vaini.
Closest hotel in vaini is Dandeli Mist Jungle Stay.
--------------------------
Retrieving results for Index 176: nador.
Closest hotel in nador is Hotel Marchica Lagoon Resort, Nador Morocco.
--------------------------
Retrieving results for Index 268: dingle.
Missing field/result... skipping.
--------------------------
Retrieving results for Index 494: baylo.
Missing field/result... skipping.
--------------------------
Retrieving results for Index 518: bonito.
Closest hotel in bonito is Zagaia Eco Resort.
--------------------------
Retrieving results for Index 527: tomatlan.
Closest hotel in tomatlan is Hotel Hacienda Vieja.
--------------------------
-------End of Search------


In [12]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
88,cidreira,BR,-30.17,-50.22,Hotel Castelo
125,arraial do cabo,BR,-22.97,-42.02,Ocean View Hotel
139,vaini,IN,15.34,74.49,Dandeli Mist Jungle Stay
176,nador,MA,35.17,-2.93,"Hotel Marchica Lagoon Resort, Nador Morocco"
268,dingle,PH,11.00,122.67,
494,baylo,PH,7.85,126.06,
518,bonito,BR,-21.13,-56.49,Zagaia Eco Resort
527,tomatlan,MX,19.93,-105.25,Hotel Hacienda Vieja


In [13]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Display figure
